# Tasks
Inputs:
1. CSV file of 64 targets that Yufei found
2. Filtered PSM files from GL261 SILAC coculture (Experiment 1) and Baseline Studies (Experiment 5) (ALL CSV files as well).
3. PSMs file from immunopeptidomics run of GL261 and CT2A.
Questions to answer
Among the possible 64 targets, for each immune cell sample in each run
1. Which peptides are in such sample, and do they have precursor abundance that is not NaN?
2. For each target peptide with precursor abundance, what is the percent rank of this abundance among all peptides that has precursor abundance?

For the tumor cell samples:
1. What peptide targets are detected, and do they have precursor abundance?
2. Among those with precursor abundance, what is the percent rank?

Then concantenate the result into a dataframe, classified by type, and export to excel.

**NOTE**: Please always create a new code cell to analyze your data to preserve the modularity of the functions.
You can always modify the functions to your need, and this will prevent the time cost when debugging. Plz (struggling undergrad)

In [1]:
# All imports are localized to this code cell to make it easier to manage dependencies.
import os
import sys
import json
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import seaborn as sns

In [2]:
#specify filtering parameters, all set to default, but can change accordingly
min_length = 8
max_length = 12
max_length2 = 30
qvalue = 0.01
search_rank = 1
type = "Mascot"

In [3]:
# Code provided by Yufei Cui for filtering MHC Data - both returns a dataframe of filtered data.
def PSM_processing_Mascot(psm_table, Ion_score, search_rank):
    """Use for files searched using Mascot"""
    peptide_length=np.zeros(len(psm_table['Sequence']))
    
    for i in range(0,len(psm_table['Sequence'])):
        peptide_length[i]=len(list(psm_table['Sequence'][i]))
    
    psm_table['Seq_len']=peptide_length
    # Filter out high quality psms
    high_rank=psm_table[psm_table['Search Engine Rank']==search_rank]
    high_qvalue=high_rank[high_rank['Ions Score']>=Ion_score]
    
    return(high_qvalue)

def PSM_processing(psm_table,min_length, max_length, qvalue):
    """Use for files searched using Sequest"""
    peptide_length=np.zeros(len(psm_table['Sequence']))
    
    for i in range(0,len(psm_table['Sequence'])):
        peptide_length[i]=len(list(psm_table['Sequence'][i]))
    psm_table['Seq_len']=peptide_length
    
    # Filter out high quality psms
    high_qvalue=psm_table[psm_table['Percolator q-Value']<=qvalue]
    long_length=high_qvalue[high_qvalue['Seq_len']>=min_length]
    good_length=long_length[long_length['Seq_len']<=max_length]
    
    return(good_length)

In [28]:
# DATA PREPROCESSING UNIT (only call functions from this cell, do not process input in this cell)
## This cell contains all the functionalities to process all the inputs, and transform them into
## a format that can be used for the rest of the analysis.
### Input: a bunch of filtered PSM files from experiments
### Output: a dictionary with sample conditions as keys, and the corresponding dataframes as values

def read_data(file_names):
    """
    Input: a list of csv filenames
    Output: a dictionary of dataframes where each key is of the form:
    (condition, mhc_type, cell_type) and the value is the initial pandas dataframe.
    """
    output = {}
    for file_name in file_names:
        if not os.path.exists(file_name):
            raise FileNotFoundError(f"File {file_name} does not exist")
        else:
            # Extract condition, cell type and mhc type from the file name, which is named beforehand for consistency
            cell_type, condition, mhc_type = file_name[:-4].split("_")
            print(f"Reading data from {file_name} with condition {condition}, cell type {cell_type}, and MHC type {mhc_type}")
            data = pd.read_csv(file_name, sep = "\t", engine = "python")
            
            # Add the data to the dictionary
            output[(condition, mhc_type, cell_type)] = data
    return output

def filter_mhc_data(data, min_length = 8, max_length = 12, qvalue = 0.01, search_rank = 1, type = "Mascot"):
    """
    Inputs: 
    data - a dictionary of pandas dataframe, where key is the condition, and value is the dataframe.
    min_length - minimum length of peptide to be considered
    max_length - maximum length of peptide to be considered
    qvalue - maximum qvalue to be considered
    search_rank
    type - type of search engine used to generate the data ("Mascot" or "Sequest")   
    Output: a dictionary of pandas dataframe, where key is the condition, and value is the filtered dataframe
    that is also grouped by sequence, and modifications are aggregated into a tuple.
    """
    if type not in ["Mascot", "Sequest"]:
        raise ValueError("Invalid search engine type! Please use either 'Mascot' or 'Sequest'")
    output = {}
    if (type == "Mascot"):
        for key in data.keys():
            psm_table = data[key]
            good_length_df = PSM_processing_Mascot(psm_table, 15, 1)
            grouped_df = good_length_df.groupby("Sequence").agg({
                'Modifications': tuple,
                **{col: 'first' for col in good_length_df.columns if col not in ['Sequence', 'Modifications']}
            }).reset_index()
            output[key] = grouped_df
    else:
        for key in data.keys():
            psm_table = data[key]
            good_length_df = PSM_processing(psm_table, min_length, max_length, qvalue)
            grouped_df = good_length_df.groupby("Sequence").agg({
                'Modifications': tuple,
                **{col: 'first' for col in good_length_df.columns if col not in ['Sequence', 'Modifications']}
            }).reset_index()
            output[key] = grouped_df
    return output 

def assert_non_empty_dataframes(data):
    """Check whether all values in the dictionary are non-empty dataframes"""
    for key, df in data.items():
        assert isinstance(df, pd.DataFrame), f"Value for key '{key}' is not a DataFrame"
        assert not df.empty, f"DataFrame for key '{key}' is empty"

def data_preprocessor(working_directory, min_length = 8, max_length = 12, qvalue = 0.01, search_rank = 1, type = "Mascot"):
    """
    Input: 
    working_directory - the directory where the data files are stored
    filenames - a list of csv filenames
    min_length - minimum length of peptide to be considered
    max_length - maximum length of peptide to be considered
    qvalue - maximum qvalue to be considered
    search_rank
    type - type of search engine used to generate the data ("Mascot" or "Sequest")
    
    Output: 
    a dictionary of pandas dataframe, where key is the condition, and value is the filtered dataframe
    that is also grouped by sequence, and modifications are aggregated into a tuple.
    """
    os.chdir(working_directory)
    current_directory = os.getcwd()
    assert current_directory == working_directory, f"Working directory is not set correctly, it is {current_directory}, not {working_directory}!"

    filenames = os.listdir(current_directory)
    assert len(filenames) > 0, "No files found in the given directory"
    data = read_data(filenames)
    #Perform checks after reading data into dictionary
    assert len(data) > 0, "No data found in the given directory"
    assert len(data) == len(filenames), "Some files are not processed or handled wrong by code"
    try:
        assert_non_empty_dataframes(data)
        print("All dataframes in the dictionary are non-empty.")
    except AssertionError as e:
        print(f"Assertion failed: {e}")
    #Filter data
    filtered_data = filter_mhc_data(data, min_length, max_length, qvalue, search_rank, type)
    return filtered_data


In [6]:
#Put target peptide data here
peptide_df_name = "/Users/asleepybeaver/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Undergraduate School/Coding projects/GitHub Management/BMDC-Cross-Presentation-Project/Data Analysis/target_peptides.csv"#FILL
target_peptides_df = pd.read_csv(peptide_df_name)
target_peptides_df

,Unnamed: 0,Sequence,Protein.Accessions,Protein,Strong Binder,Weak binder
0,1,LMPFLAEEL,Q3U2A8,Vars2,NaN,Db
1,2,RTYTYEKL,Q02248,Ctnnb1,Kb,NaN
2,3,SQLHSLTHF,Q9DB96,Ngdn,Db,Kb
3,4,ATLVFHNL,P42227,Stat3,Kb,NaN
4,5,GAPYGNPKNM,Q8BGZ2,Fam168a,Db,NaN
...,...,...,...,...,...,...
59,60,AGPTFNYL,Q01320,Top2a,Kb,Db
60,61,AAIIPYISGL,Q8CIP3,Mrgprx1,NaN,"Db,Kb"
61,62,FAYEGRDYI,P14431; P14429; P14430; P01899,H2-Q9;H2-Q7; H2-Q8; H2-D1,Db,NaN
62,63,RGFSQGEKI,Q8VCK3,Tubg2,Kb,Db


In [29]:
#For example, I do all of my data loading and preprocessing in this code cell.
# Immune cells data
immune_data_directory = "/Users/asleepybeaver/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Undergraduate School/Coding projects/GitHub Management/BMDC-Cross-Presentation-Project/Data Analysis/Immune Cell Data"
tumor_cell_directory = "/Users/asleepybeaver/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Undergraduate School/Coding projects/GitHub Management/BMDC-Cross-Presentation-Project/Data Analysis/Tumor cell data"

immune_cell_preprocessed_data = data_preprocessor(immune_data_directory) #Add other parameters if needed!
tumor_cell_preprocessed_data = data_preprocessor(tumor_cell_directory, type = "Sequest") #Add other parameters if needed!

Reading data from DC_03GL261coculture_MHCI.txt with condition 03GL261coculture, cell type DC, and MHC type MHCI
Reading data from Mac_05UT_MHCII.txt with condition 05UT, cell type Mac, and MHC type MHCII
Reading data from DC_05GL261Fascoculture_MHCI.txt with condition 05GL261Fascoculture, cell type DC, and MHC type MHCI
Reading data from DC_05GL261coculture_MHCI.txt with condition 05GL261coculture, cell type DC, and MHC type MHCI
Reading data from Mac_05GL261coculture_MHCII.txt with condition 05GL261coculture, cell type Mac, and MHC type MHCII
Reading data from Mac_05CT2Acoculture_MHCII.txt with condition 05CT2Acoculture, cell type Mac, and MHC type MHCII
Reading data from DC_05CT2Acoculture_MHCI.txt with condition 05CT2Acoculture, cell type DC, and MHC type MHCI
Reading data from DC_03GL261coculture_MHCII.txt with condition 03GL261coculture, cell type DC, and MHC type MHCII
Reading data from DC_05UT_MHCI.txt with condition 05UT, cell type DC, and MHC type MHCI
Reading data from Mac_05

In [46]:
print(tumor_cell_preprocessed_data.keys())

dict_keys([('replicate2', 'MHCI', 'CT2A'), ('replicate2', 'MHCI', 'GL261'), ('replicate1', 'MHCI', 'CT2A'), ('replicate1', 'MHCI', 'GL261')])


In [52]:
# DATA ANALYSIS UNIT (please instantiate a new code cell with all your input ready from above to run functions from this cell on)
## This cell contains all the functionalities to analyze the data.
### Input: a dictionary with sample conditions as keys, and corresponding dataframes as values
### Output: a dictionary with sample conditions as keys, and for each key, the value is a dataframe
### of size n_peptides x n_features, where features are precursor abundance and percent rank.

def run_analysis(filtered_immune_data, target_peptides_df, filtered_tumor_data = None, output_directory = None):
    """Main body function that processes data"""
    assert filtered_immune_data is not None, "No immune data provided"

    def immune_cell_helper():
        """Helper functions that first handles the immune cell data"""
        output = {}
        for (condition, mhc_type, cell_type), df in filtered_immune_data.items():
            # Filter target peptides
            print(f"Processing immune cell data for condition {condition}, MHC type {mhc_type}, and cell type {cell_type}")
            df = df.dropna(subset = ["Precursor Abundance"])
            df.loc[:, "Precursor Abundance Percent Rank"] = df["Precursor Abundance"].rank(pct = True)
            target_peptides_rank = df[df['Sequence'].isin(target_peptides_df["Sequence"])][['Sequence', 'Precursor Abundance', 'Precursor Abundance Percent Rank']]
            sorted_target_peptides_rank = target_peptides_rank.sort_values(by = "Precursor Abundance Percent Rank")
            print(f"number of peptides with rank is sorted_target_peptides_rank: {sorted_target_peptides_rank.shape[0]}")
            output[(condition, mhc_type, cell_type)] = sorted_target_peptides_rank


        return output
        
    def tumor_cell_helper():
        """Helper functions that handles the tumor cell data"""
        output = {}
        for (condition, mhc_type, cell_type), df in filtered_tumor_data.items():
            print(f"Processing tumor cell data for condition {condition}, MHC type {mhc_type}, and cell type {cell_type}")
            # Filter target peptides
            df = df.dropna(subset = ["Precursor Abundance"])
            df.loc[:, "Precursor Abundance Percent Rank"] = df["Precursor Abundance"].rank(pct = True)
            target_peptides_rank = df[df['Sequence'].isin(target_peptides_df["Sequence"])][['Sequence', 'Precursor Abundance', 'Precursor Abundance Percent Rank']]
            sorted_target_peptides_rank = target_peptides_rank.sort_values(by = "Precursor Abundance Percent Rank")
            print(f"number of peptides with rank is sorted_target_peptides_rank: {sorted_target_peptides_rank.shape[0]}")
            output[(condition, mhc_type, cell_type)] = sorted_target_peptides_rank
        print(output.keys())
        return output
    
    # Function to process and write dictionary of dataframes to a single CSV
    def write_dict_to_csv(data_dict, output_filename):
        if output_directory is not None:
            output_filename = os.path.join(output_directory, output_filename)
        # List to store processed dataframes
        dfs = []
        
        # Process each dataframe in the dictionary
        for condition, df in data_dict.items():
            # Add a 'Condition' column to the dataframe
            df['Condition'] = condition[0]
            df['MHC_Type'] = condition[1]
            df['Cell_Type'] = condition[2]
            dfs.append(df)
        
        # Concatenate all dataframes
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Write the combined dataframe to CSV
        combined_df.to_csv(output_filename, index=False)
        print(f"Written to {output_filename}")

    immune_data = immune_cell_helper()
    tumor_data = tumor_cell_helper()
    # Write immune_data to CSV
    write_dict_to_csv(immune_data, 'immune_data.csv')
    # Write tumor_data to CSV
    write_dict_to_csv(tumor_data, 'tumor_data.csv')

In [53]:
run_analysis(immune_cell_preprocessed_data, target_peptides_df, filtered_tumor_data = tumor_cell_preprocessed_data, output_directory = "/Users/asleepybeaver/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Undergraduate School/Coding projects/GitHub Management/BMDC-Cross-Presentation-Project/Data Analysis")

Processing immune cell data for condition 03GL261coculture, MHC type MHCI, and cell type DC
number of peptides with rank is sorted_target_peptides_rank: 32
Processing immune cell data for condition 05UT, MHC type MHCII, and cell type Mac
number of peptides with rank is sorted_target_peptides_rank: 0
Processing immune cell data for condition 05GL261Fascoculture, MHC type MHCI, and cell type DC
number of peptides with rank is sorted_target_peptides_rank: 11
Processing immune cell data for condition 05GL261coculture, MHC type MHCI, and cell type DC
number of peptides with rank is sorted_target_peptides_rank: 19
Processing immune cell data for condition 05GL261coculture, MHC type MHCII, and cell type Mac
number of peptides with rank is sorted_target_peptides_rank: 0
Processing immune cell data for condition 05CT2Acoculture, MHC type MHCII, and cell type Mac
number of peptides with rank is sorted_target_peptides_rank: 0
Processing immune cell data for condition 05CT2Acoculture, MHC type MHC

/var/folders/1h/csrzdv_d3ld9gnnymj2467vm0000gn/T/ipykernel_48720/1194902401.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Precursor Abundance Percent Rank"] = df["Precursor Abundance"].rank(pct = True)
/var/folders/1h/csrzdv_d3ld9gnnymj2467vm0000gn/T/ipykernel_48720/1194902401.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Precursor Abundance Percent Rank"] = df["Precursor Abundance"].rank(pct = True)
/var/folders/1h/csrzdv_d3ld9gnnymj2467vm0000gn/T/ipykernel_48720/1194902